# MMEditing 基础教程

欢迎来到MMEditing！ 这是 MMEditing 的官方 Colab 教程。在本教程中，您将学习如何使用 MMEditing 中提供的 API 训练和测试恢复器。

这是训练和测试现有模型的快速指南。如果您想基于 MMEditing 开发自己的模型并了解有关代码结构的更多信息，请参阅我们的[综合教程]()。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-mmlab/mmedit/blob/main/demo/restorer_basic_tutorial.ipynb)



## 安装MMEditing

MMEditing 可以分三步安装：

1. 安装兼容的 PyTorch 版本（你需要使用 `nvcc -V` 检查你的 CUDA 版本）。
2. 安装预编译的MMCV
3. 克隆并安装MMEditing

步骤如下所示：

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version (MMEditing needs gcc >= 5.0)
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# 安装 openmim 用于安装 mmcv-full
!pip install openmim

# 安装 mmcv-full 使用 CUDA 算子
!mim install mmcv-full

# 克隆 MMEditing
!rm -rf mmediting
!git clone https://github.com/open-mmlab/mmediting.git
%cd mmediting

# 安装 MMEditing
!pip install -v -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████▌        | 834.1MB 1.3MB/s eta 0:03:50tcmalloc: large alloc 1147494400 bytes == 0x56458d07a000 @  0x7fce190c6615 0x5645535bfcdc 0x56455369f52a 0x5645535c2afd 0x5645536b3fed 0x564553636988 0x5645536314ae 0x5645535c43ea 0x5645536367f0 0x5645536314ae 0x5645535c43ea 0x56455363332a 0x5645536b4e36 0x564553632853 0x5645536b4e36 0x564553632853 0x5645536b4e36 0x564553632853 0x5645536b4e36 0x5645537373e1 0x5645536976a9 0x564553602cc4 0x5645535c3559 0x5645536374f8 0x5645535c430a 0x5645536323b5 0x5645536317ad 0x5645535c43ea 0x5645536323b5 0x5645535c430a 0x5645536323b5
     |█████████████████████████████▊  | 1055.7MB 1.2MB/s eta 0:01:07tcmalloc: large alloc 1434370048 bytes == 0x5645d16d0000 @  0x7fce190c6615 0x5645535bfcdc 0x56455369f52a 0x5645535c2afd 0x5645536b3fed 0x564553636988 0x5645536314ae 0x5645535c43ea 0x5645536367f0 0x5645536314ae 0x5645535c43ea 0x56455363332a 0x5645536b4e36 0x564553632853 0

## 下载此演示所需的材料
在这个演示中，我们将需要一些数据和配置文件。我们将下载并放入 `./demo_files/`

In [ ]:
!wget https://download.openmmlab.com/mmediting/demo_files.zip  # 下载文件
!unzip demo_files  # 解压

# 将数据复制到 data/Set5 以备后用
!mkdir data
!mkdir data/val_set5
!cp -r demo_files/lq_images data/val_set5/Set5_bicLRx4
!cp -r demo_files/gt_images data/val_set5/Set5

--2021-07-01 11:59:48--  https://download.openmmlab.com/mmediting/demo_files.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19215781 (18M) [application/zip]
Saving to: ‘demo_files.zip’

demo_files.zip      100%[===================>]  18.33M  6.00MB/s    in 3.1s    

2021-07-01 11:59:52 (6.00 MB/s) - ‘demo_files.zip’ saved [19215781/19215781]

Archive:  demo_files.zip
   creating: demo_files/
  inflating: demo_files/demo_config_EDVR.py  
  inflating: demo_files/demo_config_BasicVSR.py  
   creating: demo_files/lq_sequences/
   creating: demo_files/lq_sequences/calendar/
  inflating: demo_files/lq_sequences/calendar/00000006.png  
  inflating: demo_files/lq_sequences/calendar/00000007.png  
  inflating: demo_files/lq_sequences/calendar/00000010.png  
  inflating: demo_files/lq_sequences/calendar/00000004.png  


## 使用预训练的图像恢复器进行推理
您可以使用 “restoration_demo.py” 轻松地使用预训练的恢复器对单个图像进行推理。您需要的是

1. `CONFIG_FILE`：你要使用的 restorer 对应的配置文件。它指定您要使用的模型。
2. `CHECKPOINT_FILE`：预训练模型权重文件的路径。
3. `IMAGE_FILE`：输入图像的路径。
4. `SAVE_FILE`：您要存储输出图像的位置。
5. `imshow`：是否显示图片。（可选的）
6. `GPU_ID`：您想使用哪个 GPU。（可选的）

获得所有这些详细信息后，您可以直接使用以下命令：

```
python demo/restoration_demo.py ${CONFIG_FILE} ${CHECKPOINT_FILE} ${IMAGE_FILE} ${SAVE_FILE} [--imshow] [--device ${GPU_ID}]
```

**注:** 
1. 配置文件位于 `./configs`。
2. 我们支持从 url 加载权重文件。您可以到相应页面（例如[这里](https://github.com/open-mmlab/mmediting/tree/master/configs/restorers/esrgan)）获取预训练模型的url。

---

我们现在将使用 `SRCNN` 和 `ESRGAN` 作为示例。



In [ ]:
# SRCNN
!python demo/restoration_demo.py ./configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth ./demo_files/lq_images/bird.png ./outputs/bird_SRCNN.png

# ESRGAN
!python demo/restoration_demo.py ./configs/restorers/esrgan/esrgan_x4c64b23g32_g1_400k_div2k.py https://download.openmmlab.com/mmediting/restorers/esrgan/esrgan_x4c64b23g32_1x16_400k_div2k_20200508-f8ccaf3b.pth ./demo_files/lq_images/bird.png ./outputs/bird_ESRGAN.png

# 检查图像是否已保存
!ls ./outputs

Use load_from_http loader
Downloading: "https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth" to /root/.cache/torch/hub/checkpoints/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth
100% 83.9k/83.9k [00:00<00:00, 1.59MB/s]
2021-07-01 12:00:10,779 - mmedit - INFO - Use load_from_torchvision loader
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100% 548M/548M [00:07<00:00, 76.0MB/s]
Use load_from_http loader
Downloading: "https://download.openmmlab.com/mmediting/restorers/esrgan/esrgan_x4c64b23g32_1x16_400k_div2k_20200508-f8ccaf3b.pth" to /root/.cache/torch/hub/checkpoints/esrgan_x4c64b23g32_1x16_400k_div2k_20200508-f8ccaf3b.pth
100% 196M/196M [00:26<00:00, 7.61MB/s]
bird_ESRGAN.png  bird_SRCNN.png


## 使用预训练的视频复原器进行推理

MMEditing 也支持视频超分辨率方法，过程类似。您可以使用带有以下参数的 `restoration_video_demo.py`：

1. `CONFIG_FILE`：你要使用的 restorer 对应的配置文件。它指定您要使用的模型。
2. `CHECKPOINT_FILE`：预训练模型权重文件的路径。
3. `INPUT_DIR`: 包含视频帧的目录。
4. `OUTPUT_DIR`: 要存储输出帧的位置。
5. `WINDOW_SIZE`: 使用滑动窗口方法时的窗口大小（可选）。
6. `GPU_ID`: 您想使用哪个 GPU（可选）。

```
python demo/restoration_video_demo.py ${CONFIG_FILE} ${CHECKPOINT_FILE} ${INPUT_DIR} ${OUTPUT_DIR} [--window_size=$WINDOW_SIZE] [--device ${GPU_ID}]
```
**注:** 视频超分辨率有两种不同的框架：***滑动窗口***和***循环***框架。使用 EDVR 等滑动窗口框架的方法时，需要指定 `window_size`。此值取决于您使用的模型。

---

我们现在将使用 `EDVR` 和 `BasicVSR` 作为示例。



In [ ]:
# EDVR（滑动窗口框架）
!python demo/restoration_video_demo.py ./configs/restorers/edvr/edvrm_wotsa_x4_g8_600k_reds.py https://download.openmmlab.com/mmediting/restorers/edvr/edvrm_wotsa_x4_8x4_600k_reds_20200522-0570e567.pth demo_files/lq_sequences/calendar/ ./outputs/calendar_EDVR --window_size=5

# BasicVSR（循环框架）
!python demo/restoration_video_demo.py ./configs/restorers/basicvsr/basicvsr_reds4.py https://download.openmmlab.com/mmediting/restorers/basicvsr/basicvsr_reds4_20120409-0e599677.pth demo_files/lq_sequences/calendar/ ./outputs/calendar_BasicVSR

# 检查是否保存了视频帧
!ls ./outputs/calendar_BasicVSR

Use load_from_http loader
Downloading: "https://download.openmmlab.com/mmediting/restorers/edvr/edvrm_wotsa_x4_8x4_600k_reds_20200522-0570e567.pth" to /root/.cache/torch/hub/checkpoints/edvrm_wotsa_x4_8x4_600k_reds_20200522-0570e567.pth
100% 11.5M/11.5M [00:01<00:00, 8.55MB/s]
2021-07-01 12:01:09,689 - mmedit - INFO - Use load_from_http loader
Downloading: "https://download.openmmlab.com/mmediting/restorers/basicvsr/spynet_20210409-c6c1bd09.pth" to /root/.cache/torch/hub/checkpoints/spynet_20210409-c6c1bd09.pth
100% 5.50M/5.50M [00:00<00:00, 8.88MB/s]
Use load_from_http loader
Downloading: "https://download.openmmlab.com/mmediting/restorers/basicvsr/basicvsr_reds4_20120409-0e599677.pth" to /root/.cache/torch/hub/checkpoints/basicvsr_reds4_20120409-0e599677.pth
100% 24.1M/24.1M [00:02<00:00, 8.97MB/s]
The model and loaded state dict do not match exactly

missing keys in source state_dict: step_counter

00000000.png  00000003.png  00000006.png  00000009.png
00000001.png  00000004.png  00

## 使用配置文件在预定义的数据集上进行测试

上述演示提供了一种对单个图像或视频序列进行推理的简单方法。如果要对一组图像或序列进行推理，可以使用位于 `./configs` 中的配置文件。
 
现有的配置文件允许您对常见数据集进行推理，例如图像超分辨率中的 `Set5` 和视频超分辨率中的 `REDS4`。您可以使用以下命令：

1. `CONFIG_FILE`: 你要使用的复原器和数据集对应的配置文件
2. `CHECKPOINT_FILE`: 预训练模型权重文件的路径。
3. `GPU_NUM`: 用于测试的 GPU 数量。
4. `RESULT_FILE`: 输出结果 pickle 文件的路径。（可选）
5. `IMAGE_SAVE_PATH`: 要存储输出图像的位置。（可选）

```
# 单 GPU 测试
python tools/test.py ${CONFIG_FILE} ${CHECKPOINT_FILE} [--out ${RESULT_FILE}] [--save-path ${IMAGE_SAVE_PATH}]

# 多 GPU 测试
./tools/dist_test.sh ${CONFIG_FILE} ${CHECKPOINT_FILE} ${GPU_NUM} [--out ${RESULT_FILE}] [--save-path ${IMAGE_SAVE_PATH}]
```
您需要做的是修改配置文件中的 `lq_folder` 和 `gt_folder`：
```
test=dict(
    type=val_dataset_type,
    lq_folder='data/val_set5/Set5_bicLRx4',
    gt_folder='data/val_set5/Set5',
    pipeline=test_pipeline,
    scale=scale,
    filename_tmpl='{}'))
```

**注**: 某些数据集类型（例如 `SRREDSDataset`）需要一个注释文件来指定数据集的详细信息。更多细节请参考 `./mmedit/dataset/` 中的相应文件。

---

以下是 SRCNN 的命令。对于其他模型，您可以简单地更改配置文件和预训练模型的路径。


In [ ]:
# 单 GPU
!python tools/test.py ./configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth --save-path ./outputs/

# 多 GPU
!./tools/dist_test.sh ./configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth 1 --save-path ./outputs/

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/registry.py", line 51, in build_from_cfg
    return obj_cls(**args)
  File "/content/mmediting/mmedit/datasets/sr_folder_dataset.py", line 62, in __init__
    self.data_infos = self.load_annotations()
  File "/content/mmediting/mmedit/datasets/sr_folder_dataset.py", line 73, in load_annotations
    lq_paths = self.scan_folder(self.lq_folder)
  File "/content/mmediting/mmedit/datasets/base_sr_dataset.py", line 39, in scan_folder
    images = list(scandir(path, suffix=IMG_EXTENSIONS, recursive=True))
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/path.py", line 63, in _scandir
    for entry in os.scandir(dir_path):
FileNotFoundError: [Errno 2] No such file or directory: 'data/val_set5/Set5_bicLRx4'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "tools/test.py", line 136, in <module>
    main()
  File "tools/test.py", line

## 在自定义数据集上进行测试

当您想在自定义数据集上进行测试时，除了数据集路径之外，您还需要修改 `test_dataset_type`。 

- 对于图像超分辨率，需要使用 `SRFolderDataset`
- 对于视频超分辨率的滑动窗口框架（例如 EDVR、TDAN），需要使用 `SRFolderVideoDataset`。
- 对于视频超分辨率的循环框架（例如 BasicVSR、IconVSR），需要使用 `SRFolderMultipleGTDataset`。

这些数据集类型假定指定目录中的所有图像/序列都用于测试。文件夹结构应该是
```
| lq_root
  | sequence_1
    | 000.png
    | 001.png
    | ...
  | sequence_2
    | 000.png
    | ...
  | ...
| gt_root
  | sequence_1
    | 000.png
    | 001.png
    |...
  | sequence_2
    | 000.png
    | ...
  | ...
```
我们将使用 **SRCNN**、**EDVR**、**BasicVSR** 作为示例。请注意 `test_dataset_type` 和 `data['test']` 的设置。

**SRCNN**

In [ ]:
# 单 GPU（Colab 只有一个 GPU）
!python tools/test.py ./demo_files/demo_config_SRCNN.py https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth --save-path ./outputs/testset_SRCNN

# 检查输出文件夹
!ls ./outputs/testset_SRCNN

Use load_from_http loader
[>>] 5/5, 8.6 task/s, elapsed: 1s, ETA:     0s
Eval-PSNR: 28.433974369836108
Eval-SSIM: 0.8099053586583066
baby.png  bird.png  butterfly.png  head.png  woman.png


**EDVR**

In [ ]:
# 单 GPU（Colab 只有一个 GPU）
!python tools/test.py ./demo_files/demo_config_EDVR.py https://download.openmmlab.com/mmediting/restorers/edvr/edvrm_wotsa_x4_8x4_600k_reds_20200522-0570e567.pth --save-path ./outputs/testset_EDVR

# 检查输出文件夹
!ls ./outputs/testset_EDVR
!ls ./outputs/testset_EDVR/city

Use load_from_http loader
[>>] 22/22, 2.0 task/s, elapsed: 11s, ETA:     0s
Eval-PSNR: 23.89569862011228
Eval-SSIM: 0.7667098470108678
calendar  city
00000000.png  00000003.png  00000006.png  00000009.png
00000001.png  00000004.png  00000007.png  00000010.png
00000002.png  00000005.png  00000008.png


**BasicVSR**

In [ ]:
# 单 GPU（Colab 只有一个 GPU）
!python tools/test.py ./demo_files/demo_config_BasicVSR.py https://download.openmmlab.com/mmediting/restorers/basicvsr/basicvsr_reds4_20120409-0e599677.pth --save-path ./outputs/testset_BasicVSR

# 检查输出文件夹
!ls ./outputs/testset_BasicVSR
!ls ./outputs/testset_BasicVSR/calendar

2021-07-01 12:02:07,780 - mmedit - INFO - Use load_from_http loader
Use load_from_http loader
The model and loaded state dict do not match exactly

missing keys in source state_dict: step_counter

[>>] 2/2, 0.2 task/s, elapsed: 11s, ETA:     0s
Eval-PSNR: 24.195768601433734
Eval-SSIM: 0.7828541339512978
calendar  city
00000000.png  00000003.png  00000006.png  00000009.png
00000001.png  00000004.png  00000007.png  00000010.png
00000002.png  00000005.png  00000008.png


## 在预定义的数据集上训练恢复器

MMEditing 使用分布式训练。以下命令可用于训练。如果要在我们的配置文件中指定的预定义数据集上进行训练，只需运行以下命令即可。

```
./tools/dist_train.sh ${CONFIG_FILE} ${GPU_NUM} [optional arguments]
```

有关可选参数的更多详细信息，请参阅 `tools/train.py`。

---

这是一个使用 EDVR 的示例。


In [ ]:
!./tools/dist_train.sh ./configs/restorers/edvr/edvrm_wotsa_x4_g8_600k_reds.py 1

2021-07-01 12:02:31,961 - mmedit - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, May  3 2021, 02:48:31) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0+cu110
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80

## 在自定义数据集上训练复原器

与您要在自己的数据集上进行测试的情况类似，您需要修改 `train_dataset_type`。您需要的数据集类型是相同的：

- 对于图像超分辨率，需要使用 `SRFolderDataset`
- 对于视频超分辨率的滑动窗口框架（例如 EDVR、TDAN），需要使用 `SRFolderVideoDataset`。
- 对于视频超分辨率的循环框架（例如 BasicVSR、IconVSR），需要使用 `SRFolderMultipleGTDataset`。

修改数据集类型和数据路径后。一切都准备好了。

In [ ]:
# SRCNN（图像超分辨率）
!./tools/dist_train.sh ./demo_files/demo_config_SRCNN.py 1 

2021-07-01 12:02:41,185 - mmedit - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, May  3 2021, 02:48:31) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0+cu110
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80

In [ ]:
# EDVR（视频超分辨率-滑动窗口）
!./tools/dist_train.sh ./demo_files/demo_config_EDVR.py 1 

2021-07-01 12:10:12,619 - mmedit - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, May  3 2021, 02:48:31) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0+cu110
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80

In [ ]:
# BasicVSR（视频超分辨率 - 循环）
!./tools/dist_train.sh ./demo_files/demo_config_BasicVSR.py 1 

2021-07-01 12:06:47,253 - mmedit - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, May  3 2021, 02:48:31) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0+cu110
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80

**本教程到此结束。有关更高级的用法，请参阅我们的[综合教程]()。享受使用 MMEditing 的乐趣！**